<a href="https://colab.research.google.com/github/vridhipuri/Capstone_Project/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd

/content


In [ ]:
import tensorflow as tf

In [ ]:
import keras

In [ ]:
import cv2

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import os
from tqdm import tqdm

In [ ]:
path="/content/drive/MyDrive/ImagePro"

In [ ]:
files=os.listdir(path)

In [ ]:
files.sort()

In [ ]:
image_array=[]
label_array=[]

In [ ]:
#loop thru each file in files
for i in tqdm(range(len(files))):
    #list of image in each folder
    sub_file=os.listdir(path+"/"+files[i])

    #loop thru each sub folder
    for j in range(len(sub_file)):
        file_path=path+"/"+files[i]+"/"+sub_file[j]
        image=cv2.imread(file_path)
        image=cv2.resize(image,(96,96))
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image_array.append(image)
        label_array.append(i)


100%|██████████| 25/25 [00:54<00:00,  2.19s/it]


In [ ]:
image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)
#xtrain will have 85% images and xtest will have 15% images
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [ ]:
del image_array,label_array

In [ ]:
import gc
gc.collect()

0

In [ ]:
model=Sequential()
#add pretrained model to sequential model. adding EfficientNetB0
pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False)

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
model.add(pretrained_model)

In [ ]:
#add pooling to model
model.add(layers.GlobalAveragePooling2D())

In [ ]:
#add dropout to model
#we add dropout to increase accuracy by reducing overfitting
model.add(layers.Dropout(0.3))

In [ ]:
#add dense layer as output
model.add(layers.Dense(1))

In [ ]:
model.build(input_shape=(None,96,96,3))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 3, 3, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4050852 (15.45 MB)
Trainable params: 4008829 (15.29 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________


In [ ]:
#compile model
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

In [ ]:
#create checkpoint to save best accuracy model
ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
filepath=ckp_path,
monitor="val_mae",
mode="auto",
save_best_only=True,
save_weights_only=True)

1. monitor validates mae loss to save model- saves best model with least mae
2. mode is used to save model when val_mae (validation mae) is min or max
3. it has 3 options min,max,auto
4. when val_mae is reduced model will be saved






In [ ]:
#create learning rate reducer to reduce lr when accuracy does not improve
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
factor=0.9, monitor="val_mae", mode="auto",cooldown=0,patience=5,verbose=1,min_lr=1e-6)

1. factor: when it is reduced, next lr will be 0.9 times of current
2. patience=x
3. reduce lr after x epochs when accuracy doesnt improve
4. verbose- show it after every epoch
5. min_lr- min learning rate

In [ ]:
#start training the model
Epochs=100
Batch_Size=32
#select bath size acc to your graphic card

In [ ]:
history=model.fit(X_train,
                  Y_train,
                  validation_data=(X_test,Y_test),
                  batch_size=Batch_Size,
                  epochs=Epochs,
                  callbacks=[model_checkpoint,reduce_lr]
                 )

Epoch 1/100


In [ ]:
#after training load best model
model.load_weights(ckp_path)

In [ ]:
#convert model to tensorflow lite model
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

In [ ]:
#save model
with open("model.tflite","wb") as f:
  f.write(tflite_model)

In [ ]:
#to see prediction result on test dataset
prediction_val=model.predict(X_test,batch_size=32)

54/54 [==============================] - 3s 19ms/step


In [ ]:
#print 1st 10 values
print(prediction_val[:10])

[[ 9.109907 ]
 [23.973286 ]
 [ 9.100234 ]
 [10.111344 ]
 [12.99855  ]
 [13.970478 ]
 [19.911165 ]
 [23.008375 ]
 [21.00702  ]
 [12.9983015]]


In [ ]:
#print 1st 10 values of ytest
print(Y_test[:10])

[ 9. 24.  9. 10. 13. 14. 20. 23. 21. 13.]


1. mae is on xtrain
2. val mae: xtest
if val_mae is reducing that means your model is improving